In [1]:
import json
import os
import copy

In [2]:
path = 'dest_arrange2/'

In [3]:
ip_dic = {}

suspicious_ip = set()

critical_size = 9000    # over 9000는 바로 의심

time_count_lim = 5          # (가변 가능) 타임 카운터(언제까지 잴 것인지)

standard_val = 0.2      # (가변 가능) 기준 값(어떤 값 이하면 의심할건지)

In [5]:
def add_ip(ip, size):
    global ip_dic
    if ip in ip_dic:
        if size in ip_dic[ip]:
            ip_dic[ip][size] = ip_dic[ip][size] + 1
        else:
            ip_dic[ip][size] = 1
    else:
        ip_dic[ip] = {}
        ip_dic[ip][size] = 1
            

In [8]:
def ip_dic_calc():
    global ip_dic
    for ip, size_val in ip_dic.items():
        total_packet = 0
        packet_size_count = 0
        for size_packet_count in size_val.values():
            total_packet = total_packet + size_packet_count
            packet_size_count = packet_size_count + 1
        calc_test = packet_size_count / float(total_packet)
        
        if standard_val > calc_test:
            suspicious_ip.add(int(ip))
            
    ip_dic.clear()

In [9]:
def count_ip_time(file_name):
    before_time = ""
    time_count = 0
    with open(path + file_name) as json_file:
        json_data = json.load(json_file)
        for item in json_data:
            if int(item["pkt_size"]) > critical_size:
                    suspicious_ip.add(int(item["source_ip"]))
            else:
                if before_time == "":
                    before_time = item["time_start"]
                    add_ip(item["source_ip"], item["pkt_size"])
                    
                elif before_time == item["time_start"]:
                    before_time = item["time_start"]
                    add_ip(item["source_ip"], item["pkt_size"])
                    
                else:
                    time_count = time_count + 1
                    if time_count >= time_count_lim:
                        ip_dic_calc()
                    add_ip(item["source_ip"], item["pkt_size"])